# Building RAG from Scratch (Open-source only!)
Following https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/

## Setup

In [ ]:
%pip install -r requirements.txt

## Sentence Transformers

In [54]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

## Llama CPP
In this notebook, we use the [llama-2-chat-13b-ggml model](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML), along with the proper prompt formatting.

In [55]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="/Users/s.choinyambuu/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/s.choinyambuu/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:         

## Postgres as vector database
`brew install postgresql`
```
vi ~/.bashrc
export LDFLAGS="-L/opt/homebrew/opt/postgresql@16/lib"
export CPPFLAGS="-I/opt/homebrew/opt/postgresql@16/include"
export PATH="/opt/homebrew/opt/postgresql@16/bin:$PATH
```

install pgvector https://github.com/pgvector/pgvector
```
cd /tmp
git clone --branch v0.7.3 https://github.com/pgvector/pgvector.git
cd pgvector
make
make install # may need sudo
```

In [34]:

import psycopg2

db_name = "vector_db"
host = "localhost"
password = "llama"
port = "5432"
user = "llama"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

ObjectInUse: database "vector_db" is being accessed by other users
DETAIL:  There are 2 other sessions using the database.


In [35]:
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="sansar_contract",
    embed_dim=384,  # openai embedding dimension
)


# Build an Ingestion Pipeline from Scratch

In [36]:

from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/sansar_dh_contract_en.pdf")
print(len(documents)) # pages

14


In [37]:

from llama_index.core.node_parser import SentenceSplitter
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [38]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [39]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [40]:
# generate embeddings
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [41]:
# add the llama index nodes into postgres vector db

vector_store.add(nodes)

['07f2f446-692b-4caa-ace4-a821bc428c8b',
 'f05f5a46-7de5-4ea4-8b6e-806cd31d05ce',
 '4e9c849f-a8d4-4a10-a6b6-36986136ac4e',
 '4dba10bd-5aba-4514-9fac-25ffb8f942d9',
 '270f049d-2dd9-4698-b39a-53f16c316bfc',
 '243444ae-71fe-436e-b4fe-970fdcf53954',
 '6e27b4dd-bc28-4db1-a7f6-2ba0e8d74ea5',
 'a7d81aa5-d560-4ba8-925b-6cb424d7aace',
 'ee9b6004-305e-4c8a-a648-e496e6a3023f',
 '781d6b3a-cc22-485c-b998-ccf1dc07c777',
 '9f870024-692a-4304-a43b-ac6a97f2b14b',
 '80d89fe6-148a-4b91-b186-32d5f1a1c86a',
 '192a32c5-70f5-4b12-95df-c96d6d63d07e',
 '77c472ec-a480-4039-990a-4fa62c883375']

# Build Retrieval Pipeline from Scratch

In [66]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
from llama_index.core.vector_stores import VectorStoreQuery
from typing import Any, List, Optional



class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [63]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=5
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [64]:

from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [65]:
query_str = "How does working hours of Sansar in Delivery Hero compare with other contracts typical in Germany?"
response = query_engine.query(query_str)
print(str(response))
# print(response.source_nodes[0].get_content())


llama_print_timings:        load time =    9104.81 ms
llama_print_timings:      sample time =       3.67 ms /   155 runs   (    0.02 ms per token, 42291.95 tokens per second)
llama_print_timings: prompt eval time =   23500.38 ms /  1234 tokens (   19.04 ms per token,    52.51 tokens per second)
llama_print_timings:        eval time =   19746.78 ms /   154 runs   (  128.23 ms per token,     7.80 tokens per second)
llama_print_timings:       total time =   43308.51 ms /  1388 tokens




Based on the provided contract, Sansar's working hours in Delivery Hero are relatively flexible and can vary based on operational requirements. The contract does not specify a fixed working schedule, and Sansar is expected to work excess hours and overtime as needed. This is in contrast to other contracts typical in Germany, which often have more rigid working schedules and fewer overtime hours. Additionally, the contract includes a clause that allows for short-time work in the event of a significant reduction in work caused by economic reasons or unavoidable events, which is not typically found in other contracts. Overall, Sansar's working hours in Delivery Hero appear to be more flexible than those in other typical German contracts.


In [53]:
query_str = "How much is the renumeration of Sansar and how does it compare to industry average for a data engineer with more than 5 years experience in Berlin, Germany"
response = query_engine.query(query_str)
print(str(response))
# print(response.source_nodes[0].get_content())

Llama.generate: prefix-match hit

llama_print_timings:        load time =   13721.58 ms
llama_print_timings:      sample time =       5.94 ms /   256 runs   (    0.02 ms per token, 43097.64 tokens per second)
llama_print_timings: prompt eval time =    2748.01 ms /    33 tokens (   83.27 ms per token,    12.01 tokens per second)
llama_print_timings:        eval time =   32725.01 ms /   255 runs   (  128.33 ms per token,     7.79 tokens per second)
llama_print_timings:       total time =   35587.25 ms /   288 tokens




Based on the provided contract, Sansar's remuneration is not explicitly mentioned. However, we can infer some information about the compensation package based on the context.

Firstly, the contract states that Sansar will receive a "relocation lump sum" of €7,500.00 (gross) to cover the costs of moving from Zurich, Switzerland to Berlin, Germany. This suggests that Sansar will be receiving a one-time payment of €7,500.00 to cover the costs of relocation.

Secondly, the contract mentions that Sansar will be employed as a Senior Data Engineer, and the company reserves the right to transfer Sansar to other equivalent work corresponding to their knowledge and skills at the same salary. This implies that Sansar will be receiving a salary that is commensurate with their experience and skills as a Senior Data Engineer.

Finally, the contract states that Sansar is entitled to 20 working days of statutory holiday entitlement per calendar year, in addition to 7 working days of voluntary additi